<a href="https://colab.research.google.com/github/Arghya-Bandyopadhyay30/Music-Recommendation-System/blob/main/Content_Based_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Music Recommender System**

**There are two main types of recommender systems:**
1.   Content-Based Filters
2.   Collaborative Filters

In [204]:
#Importing Basic Required Libraries
import numpy as np
import pandas as pd

In [205]:
#The dataset contains name, artist and lyrics for 57650 songs in English. 
data = pd.read_csv("songdata.csv")

data.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [206]:
#Number of Attributes/ Characteristics = 4
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57650 entries, 0 to 57649
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  57650 non-null  object
 1   song    57650 non-null  object
 2   link    57650 non-null  object
 3   text    57650 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [207]:
#The dataset does not contain any missing value. 
data.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [208]:
#57650 requires a lot of RAM so let as work with 3000 songs
data = data.sample(n=3000).reset_index(drop=True)

#Remove the \n in the 'text' attribute
data['text'] = data['text'].str.replace(r'\n', '')
data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


,artist,song,link,text
0,You Am I,How Much Is Enough,/y/you+am+i/how+much+is+enough_20222558.html,Would it stain If the lights fell down on you...
1,HIM,Right Here In My Arms,/h/him/right+here+in+my+arms_10139974.html,She's smiling like heaven is down on earth Su...
2,Kinks,Demolition,/k/kinks/demolition_20079077.html,I spy with my little eye Anything here that I...
3,Primus,Jellikit,/p/primus/jellikit_20110923.html,I just want to satisfy you I would like to re...
4,"Harry Connick, Jr.",S'wonderful,/h/harry+connick+jr/swonderful_20344858.html,S' wonderful! 'S marvelous! You should care f...
...,...,...,...,...
2995,Grateful Dead,Brokedown Palace,/g/grateful+dead/brokedown+palace_20062292.html,Fare you well my honey Fare you well my only ...
2996,Iggy Pop,Eggs On Plate,/i/iggy+pop/eggs+on+plate_20066989.html,Oh Lord I got eggs on my plate I got em Damn ...
2997,Arrogant Worms,Song Inside My Head,/a/arrogant+worms/song+inside+my+head_20275417...,There's a tune inside my head That makes me w...
2998,Hillsong,Jesus You Gave It All,/h/hillsong/jesus+you+gave+it+all_20816911.html,"Lord, I lift my voice in praise to You For th..."


### **TF-IDF a technique used for information retrieval.**

***Algorithm:***
1.   Find the score for TF and IDF for each term in the document
2.   Product of TF and IDF
3.   Product is the TF-IDF weight of that term

In [209]:
#Using TF-IDF vectorizerthat calculates the TF-IDF score for each 
#song lyric, word-by-word
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [210]:
#Lyric Matrix containing each word with its TF_IDF score
lyrics_matrix = tfidf.fit_transform(data['text'])

In [211]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity of each item with every other item in the dataset
cosineMatrix = cosine_similarity(lyrics_matrix) 

In [212]:
#50 similar songs dataset
similarities = {}

In [213]:
for i in range(len(cosineMatrix)):

    #Sort in ascending order for each element in cosineMatrix and 
    #get the last 50 indexes of the songs (most similar)
    similarIndices = cosineMatrix[i].argsort()[:-50:-1] 

    #For each song we store the amost 50 similar songs (excluding itself 
    #thus [1:]) with the details:
    #1. Value of the Cosine Matrix 
    #2. Name of the Song
    #3. Name of the Artist
    #4. Link of the Song
    
    similarities[data['song'].iloc[i]] = [(cosineMatrix[i][x], data['song'][x], 
                data['artist'][x], data['link'][x]) for x in similarIndices][1:]

In [219]:
class ContentBasedRecommender:
  #Parameterized Constructor
  def __init__(self, matrix):
    self.similarMatrix = matrix

  def _print_message(self, song, recomSong, recomNumber):
    print(f'The {recomNumber} recommended songs for \"{song}\" are:\n')
    
    for i in range(recomNumber):
      print(f"Song {i+1}: {recomSong[i][1]}")
      print(f"Artist: {recomSong[i][2]}")
      print(f"Link: {recomSong[i][3]}")
      print(f"Similarity Score: {round(recomSong[i][0], 3)}")
      print("--------------------\n")

  def recommend(self, recommendation):
    #Retrieving the Name of Song
    songName = recommendation['songName']

    #Number of recommend song
    numberSongs = recommendation['numberSongs']

    #Get the number of required similar songs from 'similarities' dataset
    recomendedSongs = self.similarMatrix[songName][:numberSongs]
    # print each item
    self._print_message(songName, recomendedSongs, numberSongs)

In [220]:
##Initiate Class
recommedations = ContentBasedRecommender(similarities)

In [221]:
name = input("Enter your Song Name: ")
num = int(input("Number of Recommended Songs: "))

Enter your Song Name: Jellikit
Number of Recommended Songs: 3


In [217]:
recommendation = {
    "songName": name,
    "numberSongs": num
}

recommedations.recommend(recommendation)

The 3 recommended songs for "Jellikit" are:

Song 1: Picture In Your Mind
Artist: Bread
Link: /b/bread/picture+in+your+mind_20024068.html
Similarity Score: 0.201
--------------------

Song 2: I Just Wanna
Artist: Kiss
Link: /k/kiss/i+just+wanna_20079937.html
Similarity Score: 0.192
--------------------

Song 3: I Want You
Artist: Elvis Costello
Link: /e/elvis+costello/i+want+you_20047601.html
Similarity Score: 0.188
--------------------

